In [1]:
%matplotlib inline

In [13]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
import time
from datetime import datetime
from scipy.stats import linregress
from pprint import pprint

In [5]:
# Create a set of random latitude and longitude combinations.

lats = np.random.uniform(low=-90.00,high=90.00,size=2000)
lngs = np.random.uniform(low=-180.00,high=180.00,size=2000)

lat_lngs = zip(lats,lngs)

lat_lngs

In [6]:
coordinates =list(lat_lngs)

In [7]:
cities =[]

for coordinate in coordinates:
    
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    if city not in cities:

        cities.append(city)

len(cities)

768

In [16]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [17]:
city_weather = requests.get(f'{url}&q=Tomball').json()
pprint(city_weather)

{'base': 'stations',
 'clouds': {'all': 0},
 'cod': 200,
 'coord': {'lat': 30.0972, 'lon': -95.6161},
 'dt': 1658196197,
 'id': 4737094,
 'main': {'feels_like': 97.7,
          'humidity': 58,
          'pressure': 1012,
          'temp': 89.62,
          'temp_max': 91.94,
          'temp_min': 86.52},
 'name': 'Tomball',
 'sys': {'country': 'US',
         'id': 2036642,
         'sunrise': 1658143973,
         'sunset': 1658193846,
         'type': 2},
 'timezone': -18000,
 'visibility': 10000,
 'weather': [{'description': 'clear sky',
              'icon': '01n',
              'id': 800,
              'main': 'Clear'}],
 'wind': {'deg': 200, 'gust': 27.63, 'speed': 16.11}}


In [19]:
print(city_weather['weather'][0]['description'])

clear sky


In [20]:
city_data = []

print('Beginning Data Retrieval     ')
print('-----------------------------')

# Create counters

record_count = 1
set_count = 1


# Loop through all the cities in the list

for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes

    if (i % 50 == 0 and i >= 50):

        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city

    city_url = url + "&q=" + city.replace(" ","+")


    # Log the url, record, and set numbers and the city

    print(f'Processing Record {record_count} of Set {set_count} | {city}')

    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | adrar
Processing Record 2 of Set 1 | yarada
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 6 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 7 of Set 1 | acandi
Processing Record 8 of Set 1 | yellowknife
Processing Record 9 of Set 1 | dalvik
Processing Record 10 of Set 1 | talnakh
Processing Record 11 of Set 1 | atuona
Processing Record 12 of Set 1 | evensk
Processing Record 13 of Set 1 | bluff
Processing Record 14 of Set 1 | bredasdorp
Processing Record 15 of Set 1 | galesong
Processing Record 16 of Set 1 | machala
Processing Record 17 of Set 1 | new norfolk
Processing Record 18 of Set 1 | rudnogorsk
Processing Record 19 of Set 1 | nikolskoye
Processing Record 20 of Set 1 | puerto ayora
Processing Record 21 of Set 1 | tuktoyaktuk
Processing Record

In [21]:
city_data_df = pd.DataFrame(city_data)

city_data_df.head(5)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Adrar,20.5022,-10.0711,92.66,26,61,9.93,MR,broken clouds
1,Yarada,17.6500,83.2667,84.52,84,75,5.75,IN,mist
2,Punta Arenas,-53.1500,-70.9167,39.31,81,0,27.63,CL,clear sky
3,Ushuaia,-54.8000,-68.3000,37.06,75,75,3.44,AR,broken clouds
4,Acandi,8.5063,-77.2778,77.31,89,17,3.00,CO,few clouds


In [22]:
new_column_order = ['City','Country','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Current Description']

city_data_df = city_data_df[new_column_order]

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Adrar,MR,20.5022,-10.0711,92.66,26,61,9.93,broken clouds
1,Yarada,IN,17.6500,83.2667,84.52,84,75,5.75,mist
2,Punta Arenas,CL,-53.1500,-70.9167,39.31,81,0,27.63,clear sky
3,Ushuaia,AR,-54.8000,-68.3000,37.06,75,75,3.44,broken clouds
4,Acandi,CO,8.5063,-77.2778,77.31,89,17,3.00,few clouds


In [24]:
# Output file
output_data_file = 'WeatherPy_Database.csv'

# export the df to csv
city_data_df.to_csv(output_data_file,index_label="City_ID")